In [ ]:
# import kagglehub

# Download latest version
# path = kagglehub.dataset_download("carlosgdcj/genius-song-lyrics-with-language-information")

# commented out to avoid downloading the dataset every time, since we have data.csv

100%|██████████| 3.04G/3.04G [00:36<00:00, 89.3MB/s]

Extracting files...


In [1]:
import pandas as pd

# data = pd.read_csv(path+"/song_lyrics.csv", skiprows=lambda x: x % 20 != 0)
data = pd.read_parquet("data.parquet")

In [2]:
data.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,processed,embedded
0,More Gangsta Music,rap,Cam'ron,2004,20419,"{""Cam\\'ron"",""Juelz Santana""}","[Intro: Juelz Santana]\nGangsta Music, part tw...",124,en,en,en,intro juelz santana\ngangsta music part two\nd...,"[-0.005642, 0.0287, -0.07117, 0.0644, -0.01938..."
1,Comedy Central,rap,Clipse,2002,20625,{Fabolous},"[Verse 1: Malice]\nSay dawg, let's not get inv...",34,en,en,en,verse 1 malice\nsay dawg lets not get involved...,"[0.01273, 0.0418, -0.0442, 0.01288, 0.005775, ..."
2,Not Give a Fuck,rap,Fabolous,2003,28037,{},I'm somethin' like a phenomenon\nBut still dum...,50,en,en,en,im somethin like a phenomenon\nbut still dumpi...,"[-0.00935, 0.0003202, -0.05396, 0.04517, -0.01..."
3,Suicide,rap,Fabolous,2008,15677,{},"[DJ Drama]\nSee, don't consider these niggas y...",72,en,en,en,dj drama\nsee dont consider these niggas your ...,"[-0.02151, 0.01006, -0.04138, 0.02261, -0.0128..."
4,Gettin Money Get Money Remix,rap,Junior M.A.F.I.A.,1996,77074,{},[Intro: The Notorious B.I.G. & Lil' Cease]\nMa...,90,en,en,en,intro the notorious big lil cease\nmafioso ma...,"[-0.00779, 0.01772, -0.03888, 0.03467, 0.00786..."


In [3]:
# Filter Data by English Songs only and make sure the content-type is a song.

filtered_lang_data = data[data['language'] == 'en']

filtered_data = filtered_lang_data[filtered_lang_data['tag'] != 'misc']

In [4]:
import re
import string
import numpy as np
from nltk.stem import SnowballStemmer


# ------------Exact match search and ranking-----------------#

stemmer = SnowballStemmer("english")

_PUNCT_TO_REMOVE = string.punctuation.replace("'", "")
_RE_BRACKET_TAGS = re.compile(r"\[.*?\]")


# ------------Removing tags from lyrics---------------------#
def preprocess_lyrics(lyric: str) -> str:
    if not isinstance(lyric, str):
        return ""

    text = lyric.lower()
    text = _RE_BRACKET_TAGS.sub("", text)
    text = text.translate(str.maketrans("", "", _PUNCT_TO_REMOVE))
    text = re.sub(r"\s+", " ", text).strip()
    return text

def add_processed_column(df, source_col: str = "lyrics"):
    if "processed" not in df.columns:
        df = df.copy()
        df["processed"] = df[source_col].apply(preprocess_lyrics)
    return df

#-----------------Using Stemmer for additional search----------------#
def stem_query(query):
    return " ".join(stemmer.stem(tok) for tok in query.split())


#------Simple Search for Exact matching with stemming as well---------#
def simple_search(query,
                  data,
                  top_k: int = 10):
    raw_pattern  = re.escape(preprocess_lyrics(query))
    stem_pattern = re.escape(stem_query(query))

    exact_hits = data[data['processed'].str.contains(raw_pattern,  regex=True)]
    stem_hits  = data[data['processed'].str.contains(stem_pattern, regex=True)]

    parts = [hits for hits in (exact_hits, stem_hits) if not hits.empty]
    if not parts: # no match at all
        return pd.DataFrame(columns=data.columns)

    merged = pd.concat(parts, ignore_index=True)

    # Normalize the popularity into a score
    if merged['views'].nunique() > 1:
        range = merged['views'].max() - merged['views'].min()
        merged['norm_views'] = (merged['views'] - merged['views'].min()) / range
    else:
        merged['norm_views'] = 1.0 # Full score

    merged = (merged
              .drop_duplicates('id')
              .sort_values('norm_views', ascending=False)
              .head(top_k))

    return merged.reset_index(drop=True)


Steps:

1. lowercase everything
2. remove punctuation
3. remove anything in brackets ([Verse 1], [Chorus], etc)
4. not sure if lemmatization/stemming is useful since we'd often want exact matches on lyrics

In [5]:
#------------------Add Processed Column------------------#
processed_data = add_processed_column(filtered_data)

In [7]:
from collections import Counter
import math


#---------------Adding Bigrams Search------------------#
def make_ngrams(tokens, n):
    ng = [tuple(tokens[i: i + n]) for i in range(0, len(tokens) - n + 1)]
    return ng

def ngram_dict(df, n):
    res = {}
    for index, row in df.iterrows():
        id_val = row['id']

        processed_text = row['processed']
        tokens = processed_text.split()

        ngrams = make_ngrams(tokens, n)

        res[id_val] = Counter(ngrams)
    return res

In [8]:
NGRAM_SIZE = 2

# id -> ngram count
id_nmap = ngram_dict(processed_data, NGRAM_SIZE)

# corpus ngram counts
corpus_counter = Counter()

for doc_ngrams in id_nmap.values():
    corpus_counter.update(doc_ngrams)

total_ngrams = sum(corpus_counter.values())
unique_ngrams = len(corpus_counter)

id_ncount = {doc_id: sum(doc_counter.values()) for doc_id, doc_counter in id_nmap.items()}

id_nunique = {doc_id: len(doc_counter) for doc_id, doc_counter in id_nmap.items()}

In [9]:
#-------------------BM25 BIGRAM SEARCH MODEL------------#

def get_top_matches(query, top_k=10):
    query = preprocess_lyrics(query)
    tokens = query.split()
    query_ngrams = make_ngrams(tokens, NGRAM_SIZE)
    V = unique_ngrams

    results = []

    for doc_id, doc_counter in id_nmap.items():
        total_ngrams_in_doc = id_ncount[doc_id]
        log_score = 0.0
        match_count = 0

        for ngram in query_ngrams:
            count = doc_counter.get(ngram, 0)
            if count > 0:
                match_count += 1
            prob = (count + 1) / (total_ngrams_in_doc + V)
            log_score += math.log(prob)

        if match_count > 0:
            results.append((doc_id, match_count, log_score))

    sorted_results = sorted(results, key=lambda x: (x[1], x[2]), reverse=True)

    return sorted_results[:top_k]

In [10]:
!pip install -U FlagEmbedding

In [6]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
# not running this because we already processed the data and have the embeddings
# docs = data['processed'].tolist()
# embeddings = model.encode(docs,
#                           batch_size=16,
#                           max_length=1024)['dense_vecs']

# data['embedded'] = list(embeddings)

pre tokenize: 100%|██████████| 16047/16047 [04:54<00:00, 54.40it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 16047/16047 [1:02:46<00:00,  4.26it/s]


In [7]:
data['embedded'].iloc[0]

array([-0.005642,  0.0287  , -0.07117 , ...,  0.02322 , -0.03293 ,
        0.00947 ], dtype=float16)

In [10]:
# data.to_parquet('data.parquet', index=False)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity


#-------------------Vector Search Model---------------------#
def find_most_similar(query, data, top_k=10):
  lyric_embedded = model.encode(query)['dense_vecs'].reshape(1,-1)
  # data['embedded'] = data['embedded'].apply(lambda x: np.fromstring(x.strip()[1:-1],dtype=float,sep=",").reshape(1,-1))
  # data['embedded'] = data['embedded'].apply(lambda x: x.reshape(1,-1))
  embedded_lyrics = data['embedded'].tolist()
  print(embedded_lyrics[0])
  similarities = cosine_similarity(lyric_embedded, embedded_lyrics)
  top_k_indices = similarities.argsort()[0][-top_k:][::-1]
  return [{"id": data.iloc[i]['id'], "score": similarities[0][i], "source": "embed"} for i in top_k_indices]

In [11]:
from collections import defaultdict

"""
Now that we have three different searches.
We need to take each of their rankings for the top 10
and cross reference these in order to get the best results.
"""

def normalize_scores(results):
    if not results:
        return []
    scores = np.array([r["score"] for r in results])
    min_score, max_score = scores.min(), scores.max()
    norm_scores = (scores - min_score) / (max_score - min_score + 1e-12)
    for i, r in enumerate(results):
        r["score"] = norm_scores[i]
    return results


#------------------------Hybrid Search--------------------------#
def hybrid_ranked_results(query, data, top_k=10, alpha=0.33, beta=0.33, gamma=0.33):
    # Run all search models
    simple = simple_search(query, data, top_k=top_k)
    # bm25 = get_top_matches(query, top_k=top_k)
    embed = find_most_similar(query, data, top_k=top_k)

    def wrap(results, model):
      if model == "simple":
          return [{"id": row["id"], "score": row["norm_views"], "source": model} for _, row in results.iterrows()]
      elif model == "bm25":
          return [{"id": r[0], "score": r[2], "source": model} for r in results if len(r) == 3]
      elif model == "embed":
          return results


    all_results = (
        normalize_scores(wrap(simple, "simple")) +
        # + normalize_scores(wrap(pd.DataFrame(bm25, columns=["id", "count", "score"]), "bm25"))
        normalize_scores(wrap(embed, "embed"))
    )

    combined_scores = defaultdict(float)

    for r in all_results:
        if r["source"] == "simple":
            combined_scores[r["id"]] += alpha * r["score"]
        elif r["source"] == "bm25":
            combined_scores[r["id"]] += beta * r["score"]
        elif r["source"] == "embed":
            combined_scores[r["id"]] += gamma * r["score"]

    sorted_ids = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    top_ids = [doc_id for doc_id, _ in sorted_ids[:top_k]]

    return data[data["id"].isin(top_ids)]




In [12]:
hybrid_ranked_results("we the best music", processed_data, alpha=0.75, beta=0.25, gamma=0.0)

[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]


,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,processed,embedded
65533,Our Radio Rocks,pop,Ant & Dec,2001,376,{},Here we go here we go here we go!\nHere we go ...,1653631,en,en,en,here we go here we go here we go\nhere we go h...,"[-0.00634, 0.0267, -0.05792, 0.04105, 0.00996,..."
84297,The British Grenadiers,pop,The Band Of The Grenadier Guards,2015,224,{},"For england, we march, For The King George, We...",2248573,en,en,en,for england we march for the king george we ob...,"[0.02048, 0.0489, -0.02344, -0.004803, -0.0224..."
103800,Nobody,rap,DJ Khaled,2017,63091,"{""Nicki Minaj"",""Alicia Keys""}",[Intro: DJ Khaled & Nicki Minaj]\nWe the Best ...,3117583,en,en,en,intro dj khaled nicki minaj\nwe the best musi...,"[0.02803, 0.06024, -0.03824, 0.02573, -0.02849..."
137877,The Greatest of All Time,rap,Brazen-Faced,2018,30,{},"(Intro)\nSkrrt, yeah we ridin’\nMotherfucker w...",4143621,en,en,en,intro\nskrrt yeah we ridin’\nmotherfucker we r...,"[-0.010704, 0.04425, -0.01235, 0.0538, -0.0612..."
153919,Expensive Car Radio,pop,Big Baller B,2019,93,{},[Intro: JAY-Z & DJ Khaled]\nAnd all that shit ...,4651353,en,en,en,intro jayz dj khaled\nand all that shit real ...,"[0.02077, 0.01562, -0.04477, 0.05328, -0.00343..."
177423,Off-White,rap,GodFearin,2020,214,{Realnya},[Chorus: Teejay Godfearing]\nAll of my shoes o...,5364498,en,en,en,chorus teejay godfearing\nall of my shoes offw...,"[0.007717, 0.03497, -0.0714, 0.0819, -0.00843,..."
190991,POPSTAR,rap,DJ Khaled,2020,1021462,{Drake},[Intro: DJ Khaled & Drake]\nBitches\nWe The Be...,5774099,en,en,en,intro dj khaled drake\nbitches\nwe the best m...,"[-0.01558, -0.02179, -0.03635, 0.00679, -0.016..."
214560,HARD BEAT,rap,The Muffler Man,2021,4,{},"[Intro]\nI be on duh street\nYeah, I be on duh...",6513323,en,en,en,intro\ni be on duh street\nyeah i be on duh st...,"[0.01842, 0.005684, -0.03024, 0.02579, -0.0084..."
214563,Dead by Dusk Vol. 1,rap,The Muffler Man,2020,2,{},[Intro]\nWe the best music for real\n\n[Chorus...,6513384,en,en,en,intro\nwe the best music for real\n\nchorus\np...,"[0.02356, 0.0214, 0.000734, -0.02454, -0.01746..."
244161,Blessed,rap,WINGZ & Walter III,2021,3,{DIRECTEDBYKIZITO},[Hook}\nWe ain’t got no stress\n(woo woo)\nHos...,7442636,en,en,en,hook\nwe ain’t got no stress\nwoo woo\nhosts w...,"[0.013214, 0.01794, -0.04913, -0.01225, -0.021..."


In [13]:
#-----------------Evaluation-----------------------#

# MAP@10
def average_precision(retrieved_docs, relevant_docs):
    score = 0.0
    hits = 0
    for i, doc_id in enumerate(retrieved_docs, 1):
        if doc_id in relevant_docs:
            hits += 1
            score += hits / i
    return score / max(1, len(relevant_docs))

def mean_average_precision(queries, relevance_dict, retriever_fn, top_k=10):
    total_ap = 0.0
    for query in queries:
        retrieved_docs = retriever_fn(query)["id"].tolist()
        relevant_docs = relevance_dict.get(query, set())
        total_ap += average_precision(retrieved_docs[:top_k], relevant_docs)
    return total_ap / len(queries)


# Recall@10
def recall_at_k(retrieved_docs, relevant_docs, k=10):
    return len(set(retrieved_docs[:k]) & relevant_docs) / len(relevant_docs)

def mean_recall(queries, relevance_dict, retriever_fn, top_k=10):
    total_recall = 0.0
    for query in queries:
        retrieved_docs = retriever_fn(query)["id"].tolist()
        relevant_docs = relevance_dict.get(query, set())
        total_recall += recall_at_k(retrieved_docs, relevant_docs, top_k)
    return total_recall / len(queries)


# TODO: Create a dataset for evaluation. Perhaps using manual annotation on a small set of queries and seeing how it compares.

In [17]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 21.9 MB/s eta 0:00:00


In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Replace with your actual Spotify API credentials
SPOTIPY_CLIENT_ID = "364006f1494e49b2b61b9c8cfca2a03d"
SPOTIPY_CLIENT_SECRET = "f7669329477443e5a3fd9ace07717698"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                           client_secret=SPOTIPY_CLIENT_SECRET))

def search_spotify(query, limit=10):
    results = sp.search(q=query, limit=limit, type='track')
    tracks = results['tracks']['items']
    song_list = []
    for track in tracks:
        song_list.append(track['name'])
    return song_list

# Example usage
search_query = "we the best music"
songs = search_spotify(search_query)
songs

['Best Thing Ever - Stage Version',
 'LOVE, MONEY, FAME (feat. DJ Khaled)',
 'We Are The Heroes',
 "We're Gonna Get Wet from Kidsongs: The 50 Best Summer Songs",
 'LOVE, MONEY, FAME (feat. DJ Khaled)',
 'FAME',
 'The Path of the Wind',
 "I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne)",
 'The Power',
 'Yes! We Have No Bananas from The 50 Best Summer Songs']

In [29]:
def test_search(query):
    spotify_songs = search_spotify(query)
    print(f"Spotify Songs:")
    for song in spotify_songs:
        print(song)
    custom_songs = hybrid_ranked_results(query, processed_data, alpha=0.6, beta=0.0, gamma=0.40)["title"].tolist()
    print(f"Our Retrieved Songs:")
    for song in custom_songs:
        print(song)
    precision = average_precision(custom_songs, list(set(spotify_songs)))
    print(f"Precision: {precision:.4f}")
    recall = recall_at_k(custom_songs, set(spotify_songs))
    print(f"Recall: {recall:.4f}")

In [30]:
search_query = "we wish you a merry christmas"
test_search(search_query)

Spotify Songs:
We Wish You a Merry Christmas
We Wish You a Merry Christmas
Sing a Christmas Carol
We Wish You a Merry Christmas - Sing Along
We Wish You A Merry Christmas
Have Yourself A Merry Little Christmas
We Wish You a Merry Christmas
We Wish You A Merry Christmas
Have Yourself A Merry Little Christmas
We Wish You a Merry Christmas
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
We Wish You a Merry Christmas
We Wish You a Merry Christmas
Have Yourself A Merry Little Christmas
Have Yourself a Merry Little Christmas
Sleigh Ride
Have Yourself A Merry Little Christmas
Have Yourself A Merry Little Christmas
Have Yourself A Merry Little Christmas
Have Yourself a Merry Little Christmas
We Wish You a Merry Christmas/God Rest Ye Merry Gentlemen/O Come All Ye Faithful/Joy to the World
Precision: 1.0262
Recall: 0.4000


In [31]:
search_query = "you will remember me for centuries"
test_search(search_query)

Spotify Songs:
Centuries (Remember Me)
Centuries
Centuries
Centuries - Gazzo Remix
Centuries - Remix
Centuries (You Will Remember Me)
Centuries
Centuries - Radio Edit
Will You
Centuries
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
Same Sweet Girl
I Will Remember You
Bark 2
For How Long
I Will Remember You Rhythm remix
Im thinking of you all the time
The Interlude
One Year and One Day
Together With Me
One Hot Day
Precision: 0.0000
Recall: 0.0000


In [32]:
search_query = "look what you made me do"
test_search(search_query)

Spotify Songs:
Look What You Made Me Do
I Like Me Better
I Did Something Bad
Look What You Made Me Do
Numb / Encore
我是真的相信過愛情
Look What You Made Me Do - Karaoke Version
Look What You Made Me Do (Sped Up Version)
Numb / Encore
Bad Blood (Taylor's Version)
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
She Did That
Fond Farewell
Wild Mushrooms
I Saw You With Your Heart Looking at Me
What Do You Want Me To Do
Black Poetry
Look What You Made Me Do
Look what u made me do
My James Dean
Look What You Made Me Do
Precision: 0.0429
Recall: 0.1250


In [33]:
search_query = "sing us a song youre the piano man"
test_search(search_query)

Spotify Songs:
Piano Man
Sing a Song of Sixpence
Piano Man
Piano Man
Your Song - Piano Version
Piano Man
Sing Yourself a Smile - 2019 Version
Piano Man
What Makes You Beautiful
Piano Man
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
Sure Thing Remix
Sing Sing Sing
The Boogie Man Boogie
Louis Louis
Boogie Man live
Piano Man
Voyage
Intro
Fishin With You
Big Man
Precision: 0.0333
Recall: 0.2000


In [34]:
search_query = "bread in my jar"
test_search(search_query)

Spotify Songs:
Jars: Быстрая бабушка (Nopea äitimuori)
Piano Man
Bread
Piano Man
Bread
Piano Man
I Am The Bread Of Life (I Will Raise Him Up)
Piano Man - Live from Radio 2's Chris Evans Breakfast Show
SUMMER BLUE
Piano Man
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
Sandwiches
Pure Jam
Piano Man
Rude Boy Jamaican
Kim Jong ILL
Cereal
The Bell Jar
Damn Handjob
Bready
Bread Pit
Precision: 0.0556
Recall: 0.1667


In [35]:
search_query = "get your game on go play"
test_search(search_query)

Spotify Songs:
All Star
Totally Transformed
All Star - Acoustic
Clear Eyes, Full Heart, Can’t Lose
All Star
Get Your Game On!
All Star
Game Time (feat. Sage the Gemini)
All Star - Owl City Remix
Your Gain
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
Signal in the Sky Lets Go acoustic demo
One More Time Tony Moran extended mix
Riding High
Play my game
Keep It Pimpin
All Star
Bring It Up Reprise Live At The Garden/1967
I Love Your Body
Its Summer
Dumbthicc
Precision: 0.0208
Recall: 0.1250


In [36]:
search_query = "happy together"
test_search(search_query)

Spotify Songs:
Happy Together
Happy Together
Happy Together (feat. Ray Toro)
Happy Together - Acoustic Version
Happy Together - Remastered
Happy Together
My Happy Song
Happy Together
Happy Together
Happy Together
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
We Match
Light Up
Sawubona
You And Me
Happy
Thats Not How the Story Goes
Feel Well
I Love Your Body
Adventures of Tricky N Duke
Together
Precision: 0.0000
Recall: 0.0000


In [37]:
search_query = "Me and you, and you and me No matter how they tossed the dice"
test_search(search_query)


Spotify Songs:
ME N U DATE
Me & U
ME / U - DELUXE EDITION
Me and You
Me And U
You and Me
ME N U DATE - Slowed & Reverbed
me & u (tiktok remix) - sped up + reverb
Me & You
Me & You
[-0.005642  0.0287   -0.07117  ...  0.02322  -0.03293   0.00947 ]
Our Retrieved Songs:
No Regrets
Lucky For Me
Deal Em Again
Juliette
Wipe The Slate Clean
Doesnt Matter
VEGAS
The Casino
Walk in the Park
How Was It for You
Precision: 0.0000
Recall: 0.0000
